In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
filepath = "../raw_data/pv_generation_data"
filename_actual = "solar_energy_extrapolation.csv"
filename_forecast = "solar_energy_forecast.csv"

In [3]:
# df = pd.read_csv(os.path.join(filepath, filename_actual), sep=';',)

# df.columns = ['date', 'from','to','50hertz(MW)','Amprion(MW)','TenneTTSO(MW)','TransnetBW(MW)']
# B_idxs = df.apply(lambda row: True if ('B' in row['from']) or ('B' in row['to']) else False, axis=1)
# df.loc[B_idxs,'to'] = df.loc[B_idxs,'to'].str.replace('02B:00','03:00')

In [4]:
# df.loc[B_idxs]

In [5]:
def clean_pv_data(df):
    df.columns = ['date', 'from','to','50hertz(MW)','Amprion(MW)','TenneTTSO(MW)','TransnetBW(MW)']
    df['from'] = df['date'].str[:] + '-' + df['from'].str[:]
    df['to'] = df['date'].str[:] + '-' + df['to'].str[:]
    # find indexes which have 'A's and 'B's
    A_idxs = df.apply(lambda row: True if ('A' in row['from']) or ('A' in row['to']) else False, axis=1)
    B_idxs = df.apply(lambda row: True if ('B' in row['from']) or ('B' in row['to']) else False, axis=1)
    # drop 'B's and set 0 to 'A's
    # remove 'AB' chars too.
    
    data_cols = ['50hertz(MW)','Amprion(MW)','TenneTTSO(MW)','TransnetBW(MW)']
    for col_name in data_cols:
        print("Cleaning {}...".format(col_name))
        df.loc[A_idxs, col_name] = 0
        
        #change dtype to numeric
        df[col_name] = df[col_name].astype(str)
        df[col_name] = df[col_name].str.replace(',','.')
        df[col_name] = df[col_name].astype(float)
        
    df.loc[A_idxs,'from'] = df.loc[A_idxs,'from'].str.replace('A','')
    df.loc[A_idxs,'to'] = df.loc[A_idxs,'to'].str.replace('A','')
    df.loc[B_idxs,'to'] = df.loc[B_idxs,'to'].str.replace('02B:00','03:00')
    B_idxs = df.apply(lambda row: True if ('B' in row['from']) or ('B' in row['to']) else False, axis=1) # find Bs again
    df.drop(B_idxs.loc[B_idxs].index, inplace=True) #drop 'B's
        
    
    
    # change dtype to datetime
    print("Processing pd.to_datetime('date')...")
    df['date'] = df['date'].apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
    print("Processing pd.to_datetime('from')...")
    df['from'] = df['from'].apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y-%H:%M'))
    print("Processing pd.to_datetime('to')...")
    df['to'] = df['to'].apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y-%H:%M'))
    
    df.reset_index(drop=True, inplace=True)
    return df

actual_file = pd.read_csv(os.path.join(filepath, filename_actual), sep=';',)
forecast_file = pd.read_csv(os.path.join(filepath, filename_forecast), sep=';')
actual_dataframe = clean_pv_data(actual_file)
forecast_dataframe = clean_pv_data(forecast_file)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Cleaning 50hertz(MW)...
Cleaning Amprion(MW)...
Cleaning TenneTTSO(MW)...
Cleaning TransnetBW(MW)...
Processing pd.to_datetime('date')...
Processing pd.to_datetime('from')...
Processing pd.to_datetime('to')...
Cleaning 50hertz(MW)...
Cleaning Amprion(MW)...
Cleaning TenneTTSO(MW)...
Cleaning TransnetBW(MW)...
Processing pd.to_datetime('date')...
Processing pd.to_datetime('from')...
Processing pd.to_datetime('to')...


In [6]:
def merge_actual_forecast(actual_dataframe, forecast_dataframe):
    df = actual_dataframe.copy()
    data_cols = ['50hertz(MW)','Amprion(MW)','TenneTTSO(MW)','TransnetBW(MW)']
    for old_colname in data_cols: 
        df.rename(columns={old_colname:'actual_'+old_colname}, inplace=True)
        df['forecast_'+old_colname] = forecast_dataframe[old_colname]
    return df

merged_df = merge_actual_forecast(actual_dataframe, forecast_dataframe)
merged_df

,date,from,to,actual_50hertz(MW),actual_Amprion(MW),actual_TenneTTSO(MW),actual_TransnetBW(MW),forecast_50hertz(MW),forecast_Amprion(MW),forecast_TenneTTSO(MW),forecast_TransnetBW(MW)
0,2011-04-01,2011-04-01 00:00:00,2011-04-01 00:15:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
1,2011-04-01,2011-04-01 00:15:00,2011-04-01 00:30:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
2,2011-04-01,2011-04-01 00:30:00,2011-04-01 00:45:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
3,2011-04-01,2011-04-01 00:45:00,2011-04-01 01:00:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
4,2011-04-01,2011-04-01 01:00:00,2011-04-01 01:15:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
5,2011-04-01,2011-04-01 01:15:00,2011-04-01 01:30:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
6,2011-04-01,2011-04-01 01:30:00,2011-04-01 01:45:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
7,2011-04-01,2011-04-01 01:45:00,2011-04-01 02:00:00,0.0,0.00,0.000,0.0,NaN,0.0,0.000,0.0
8,2011-04-01,2011-04-01 02:00:00,2011-04-01 02:15:00,0.0,0.00,0.000,0.0,0.0,0.0,0.000,0.0
9,2011-04-01,2011-04-01 02:15:00,2011-04-01 02:30:00,0.0,0.00,0.000,0.0,0.0,0.0,0.000,0.0


In [7]:
from collections import OrderedDict
def split_companies(company_name, dataframe):
    return pd.DataFrame(OrderedDict([('date',dataframe['date']),
                                     ('from',dataframe['from']),
                                     ('to',dataframe['to']),
                                     ('actual',dataframe['actual_{}(MW)'.format(company_name)]),
                                     ('forecast',dataframe['forecast_{}(MW)'.format(company_name)])]))
def write_csv(company_name, merged_df):
    split_companies(company_name, merged_df).to_csv(
        '../input/PV_Generation_{}_2011-2017_cleaned.csv'.format(company_name), index=None)

write_csv('50hertz', merged_df)
write_csv('Amprion', merged_df)
write_csv('TenneTTSO', merged_df)
write_csv('TransnetBW', merged_df)


In [8]:
last= pd.to_datetime('2017-12-31 17:15:00') 
first= pd.to_datetime('2011-04-01') 

In [40]:
h = pd.read_csv('../input/Load_Amprion_2010-2017_cleaned.csv')
h['from'] = h['from'].apply(lambda x: pd.to_datetime(x))

In [41]:
h.loc[(h['from'].diff(1) != pd.Timedelta(minutes=15))]

,date,from,to,actual,forecast
0,2010-01-01 00:00:00,2010-01-01 00:00:00,2010-01-01 00:15:00,16990.0,19211.0
96,2010-02-01 00:00:00,2010-02-01 00:00:00,2010-02-01 00:15:00,16558.0,17258.0
192,2010-03-01 00:00:00,2010-03-01 00:00:00,2010-03-01 00:15:00,18529.0,18951.0
288,2010-04-01 00:00:00,2010-04-01 00:00:00,2010-04-01 00:15:00,19038.0,16948.0
384,2010-05-01 00:00:00,2010-05-01 00:00:00,2010-05-01 00:15:00,20989.0,18042.0
480,2010-06-01 00:00:00,2010-06-01 00:00:00,2010-06-01 00:15:00,21625.0,18108.0
576,2010-07-01 00:00:00,2010-07-01 00:00:00,2010-07-01 00:15:00,21651.0,21475.0
672,2010-08-01 00:00:00,2010-08-01 00:00:00,2010-08-01 00:15:00,21833.0,20725.0
768,2010-10-01 00:00:00,2010-10-01 00:00:00,2010-10-01 00:15:00,21229.0,19381.0
864,2010-11-01 00:00:00,2010-11-01 00:00:00,2010-11-01 00:15:00,20273.0,19687.0


In [42]:
h.iloc[90:100]

,date,from,to,actual,forecast
90,2010-01-01 00:00:00,2010-01-01 22:30:00,2010-01-01 22:45:00,18576.0,19026.0
91,2010-01-01 00:00:00,2010-01-01 22:45:00,2010-01-01 23:00:00,18295.0,18658.0
92,2010-01-01 00:00:00,2010-01-01 23:00:00,2010-01-01 23:15:00,17951.0,18444.0
93,2010-01-01 00:00:00,2010-01-01 23:15:00,2010-01-01 23:30:00,17665.0,18008.0
94,2010-01-01 00:00:00,2010-01-01 23:30:00,2010-01-01 23:45:00,17139.0,17670.0
95,2010-01-01 00:00:00,2010-01-01 23:45:00,2010-01-01 00:00:00,16857.0,17274.0
96,2010-02-01 00:00:00,2010-02-01 00:00:00,2010-02-01 00:15:00,16558.0,17258.0
97,2010-02-01 00:00:00,2010-02-01 00:15:00,2010-02-01 00:30:00,16294.0,17020.0
98,2010-02-01 00:00:00,2010-02-01 00:30:00,2010-02-01 00:45:00,15981.0,16665.0
99,2010-02-01 00:00:00,2010-02-01 00:45:00,2010-02-01 01:00:00,15820.0,16426.0


In [34]:
h[70:120]

,date,from,to,actual,forecast
70,2010-01-01 00:00:00,2010-01-01 17:30:00,2010-01-01 17:45:00,7765.0,7082.0
71,2010-01-01 00:00:00,2010-01-01 17:45:00,2010-01-01 18:00:00,7856.0,7132.0
72,2010-01-01 00:00:00,2010-01-01 18:00:00,2010-01-01 18:15:00,7707.0,7643.0
73,2010-01-01 00:00:00,2010-01-01 18:15:00,2010-01-01 18:30:00,7749.0,7628.0
74,2010-01-01 00:00:00,2010-01-01 18:30:00,2010-01-01 18:45:00,7711.0,7649.0
75,2010-01-01 00:00:00,2010-01-01 18:45:00,2010-01-01 19:00:00,7688.0,7644.0
76,2010-01-01 00:00:00,2010-01-01 19:00:00,2010-01-01 19:15:00,7854.0,8101.0
77,2010-01-01 00:00:00,2010-01-01 19:15:00,2010-01-01 19:30:00,7786.0,8055.0
78,2010-01-01 00:00:00,2010-01-01 19:30:00,2010-01-01 19:45:00,7727.0,7997.0
79,2010-01-01 00:00:00,2010-01-01 19:45:00,2010-01-01 20:00:00,7607.0,7940.0


In [15]:
pd.to_datetime('2017-12-31') - pd.to_datetime('2010-01-01')

Timedelta('2921 days 00:00:00')

In [16]:
2922*4*24

280512

In [17]:
pd.to_datetime('2017-12-31') - pd.to_datetime('2011-04-01')

Timedelta('2466 days 00:00:00')

In [19]:
2467*4*24

236832